In [ ]:
!python createContentTrainingData.py --input /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/product_data/products --output /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/labeled_products.txt

In [1]:
!wc - l  /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/labeled_products.txt

wc: -: open: No such file or directory
wc: l: open: No such file or directory
  115503 1180200 9329538 /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/labeled_products.txt
  115503 1180200 9329538 total


In [96]:
!shuf /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/labeled_products.txt > shuffled_labeled_products.txt

In [3]:
!wc -l /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/shuffled_labeled_products.txt

  115503 /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/shuffled_labeled_products.txt


In [5]:
!head -n 10000 /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/shuffled_labeled_products.txt > training_data.txt
!tail -n 10000 /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/shuffled_labeled_products.txt > test_data.txt

In [18]:
# Train the model using fasttext supervised
import fasttext

product_classifier_basic = fasttext.train_supervised(input="training_data.txt", lr=1.0, epoch=25)
product_classifier_basic.predict("Whirlpool - 6.5 Cu. Ft. 7-Cycle Super Capacity Gas Dryer - Biscuit-on-Biscuit")
product_classifier_basic.test("test_data.txt")

Read 0M words
Number of words:  11114
Number of labels: 1359
Progress: 100.0% words/sec/thread:   36679 lr:  0.000000 avg.loss:  1.075883 ETA:   0h 0m 0s 11.3% words/sec/thread:   34680 lr:  0.886652 avg.loss:  6.865002 ETA:   0h 0m 4s


(9652, 0.622150849564857, 0.622150849564857)

In [20]:
# Increasing Recall
print(product_classifier_basic.test("test_data.txt", 3))
print(product_classifier_basic.test("test_data.txt", 4))
print(product_classifier_basic.test("test_data.txt", 5))

(9652, 0.2583920430998757, 0.775176129299627)
(9652, 0.20024347285536676, 0.800973891421467)
(9652, 0.1637173642768338, 0.818586821384169)


In [22]:
product_classifier_v2 = fasttext.train_supervised(input="training_data.txt", lr=1.0, epoch=300, wordNgrams=2)
product_classifier_v2.predict("Whirlpool - 6.5 Cu. Ft. 7-Cycle Super Capacity Gas Dryer - Biscuit-on-Biscuit")
product_classifier_v2.test("test_data.txt")
# response format (N, P@1, R@1)

Read 0M words
Number of words:  11114
Number of labels: 1359
Progress: 100.0% words/sec/thread:   26274 lr:  0.000000 avg.loss:  0.117212 ETA:   0h 0m 0s  7.2% words/sec/thread:   37824 lr:  0.928031 avg.loss:  1.411095 ETA:   0h 0m55s19.3% words/sec/thread:   33776 lr:  0.807063 avg.loss:  0.547273 ETA:   0h 0m53s 71.0% words/sec/thread:   28543 lr:  0.290138 avg.loss:  0.160695 ETA:   0h 0m22s76.6% words/sec/thread:   28310 lr:  0.234424 avg.loss:  0.150031 ETA:   0h 0m18s 0m13s 90.3% words/sec/thread:   27084 lr:  0.096759 avg.loss:  0.128689 ETA:   0h 0m 8s


(9652, 0.6246373808537091, 0.6246373808537091)

## Normalizing Training data

In [23]:
!cat training_data.txt |sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" | sed "s/[^[:alnum:]_]/ /g" | tr -s ' ' > normalized_training_data.txt
#That won’t give you stemming, but it will:
#Remove all non-alphanumeric characters other than underscore (which we need for the labels!).
# Convert all letters to lowercase.
# Trim excess space characters so that tokens are separated by a single space.

In [26]:
!cat test_data.txt |sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" | sed "s/[^[:alnum:]_]/ /g" | tr -s ' ' > normalized_test_data.txt

In [30]:
product_classifier_norm = fasttext.train_supervised(input="normalized_training_data.txt", lr=1.0, epoch=25)
product_classifier_norm.test("normalized_test_data.txt")

Read 0M words
Number of words:  8675
Number of labels: 1359
Progress: 100.0% words/sec/thread:   38623 lr:  0.000000 avg.loss:  0.961374 ETA:   0h 0m 0s


(9652, 0.6183174471612101, 0.6183174471612101)

In [32]:
!head -n 10000 /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/stemmed_labeled_products.txt > stem_training_data.txt
!tail -n 10000 /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/stemmed_labeled_products.txt > stem_test_data.txt

In [36]:
product_classifier_stem = fasttext.train_supervised(input="stem_training_data.txt", lr=1.0, epoch=100)
product_classifier_stem.test("stem_test_data.txt")

Read 0M words
Number of words:  8807
Number of labels: 1125
Progress: 100.0% words/sec/thread:   35028 lr:  0.000000 avg.loss:  0.213642 ETA:   0h 0m 0s7s


(9136, 0.49452714535901926, 0.49452714535901926)

## Pruning Attempt

In [40]:
filepath = '/Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/labeled_products.txt'
entries = []
with open(filepath) as fp:
   line = fp.readline()
   while line:
       vals = line.split(" ", 1)
       # print(f"{len(vals)} -> {vals[0]}-{vals[1]}")
       line = fp.readline()
       entries.append((vals[0], vals[1]))

print(len(entries))
print(entries[0])


115503
('__label__pcmcat251500050009', 'Ultimate Beginner Series: Electric Guitar Complete - DVD\n')


In [41]:
import pandas as pd
df = pd.DataFrame(entries, columns =['Category', 'Name'])

In [56]:
cat_count = df.groupby(['Category']).Name.count()
print(cat_count.size)
cat_count

1964


Category
__label__abcat0011000         12
__label__abcat0011001          8
__label__abcat0011002          2
__label__abcat0011003          8
__label__abcat0011004          1
                              ..
__label__pcmcat86300050019     1
__label__pcmcat94300050028     6
__label__pcmcat94500050010     2
__label__pcmcat97200050013     3
__label__pcmcat99000050002     6
Name: Name, Length: 1964, dtype: int64

In [62]:
# https://stackoverflow.com/questions/50824493/pandas-groupby-filter-on-count
pruned_cats_500 = cat_count[cat_count > 500].index.tolist()
print(len(pruned_cats_500))
pruned_cats_100 = cat_count[cat_count > 100].index.tolist()
print(len(pruned_cats_100))

32
262


In [93]:
# df[df['Category']=='__label__abcat0011000']
cat_count[cat_count.index==pruned_cats_500[0]]

Category
__label__abcat0101001    2100
Name: Name, dtype: int64

In [64]:
input_filepath = '/Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/labeled_products.txt'
output_filepath = '/Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/pruned_labeled_products_500.txt'

ip_file = open(input_filepath, 'r')
op_file = open(output_filepath, 'w')

line = ip_file.readline()
while line:
   print(line)
   vals = line.split(" ", 1)
   line = ip_file.readline()
   if vals[0] in pruned_cats_500:
       op_file.write(line)


ip_file.close()
op_file.close()



__label__pcmcat251500050009 Ultimate Beginner Series: Electric Guitar Complete - DVD

__label__pcmcat235500050004 Generator Rex: Agent of Providence - Nintendo 3DS

__label__abcat0701002 Generator Rex: Agent of Providence - Xbox 360

found __label__abcat0701002
__label__abcat0701002 Cabela's Adventure Camp - Xbox 360

found __label__abcat0701002
__label__abcat0701002 Cabela's Big Game Hunter Hunting Party with Top Shot Sport - Xbox 360

found __label__abcat0701002
__label__abcat0701002 Big League Sports - Xbox 360

found __label__abcat0701002
__label__abcat0707002 Bakugan: Rise of the Resistance - Nintendo DS

found __label__abcat0707002
__label__abcat0706002 DreamWorks Super Star Kartz - Nintendo Wii

found __label__abcat0706002
__label__abcat0707002 DreamWorks Super Star Kartz - Nintendo DS

found __label__abcat0707002
__label__abcat0703002 DreamWorks Super Star Kartz - PlayStation 3

found __label__abcat0703002
__label__abcat0701002 DreamWorks Super Star Kartz - Xbox 360

found __la

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [67]:
# Now shuffle and split this data as before, extracting 10000 training examples and 10000 test examples. Normalize it if you’d like. Train a model as before – keep the learning rate of 1.0 and 25 epochs that have been serving us well. Test it to evaluate P@1
# !shuf /Users/sengopal/build/my-git/search_with_machine_learning_course/datasets/fasttext/pruned_labeled_products_500.txt > shuffled_pruned_labeled_products_500.txt
!head -n 10000 shuffled_pruned_labeled_products_500.txt > prune_500_training_data.txt
!tail -n 10000 shuffled_pruned_labeled_products_500.txt > prune_500_test_data.txt

In [95]:
product_classifier_prune_500 = fasttext.train_supervised(input="prune_500_training_data.txt", lr=1.0, epoch=25,wordNgrams=3)
product_classifier_prune_500.test("prune_500_test_data.txt")

Read 0M words
Number of words:  9279
Number of labels: 996
Progress: 100.0% words/sec/thread:   52581 lr:  0.000000 avg.loss:  0.981558 ETA:   0h 0m 0s


(9602, 0.7471360133305561, 0.7471360133305561)

## (TODO) Leverage the hierarchical nature of our taxonomy and roll up infrequently used labels to their parent or other ancestor categories.